In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import scanpy.external as sce
import sys
import scipy 

In [2]:
import random

In [3]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 


In [4]:
df = pd.read_csv('raw_h1299_36h_counts.csv', sep=',')

In [5]:
conditions = []
for i in list(range(df.shape[0])):
    conditions.append(df["Unnamed: 0"][i].split('-')[1])

In [6]:
set(conditions)

{'S2', 'mock'}

## Dif expr 

In [7]:
def prep_df(filename):
    df = pd.read_csv(filename, sep=',')
    genes = list(df.columns[1:])
    counts = df.T
    counts.columns = df.iloc[:, 0]
    counts = counts.drop('Unnamed: 0', axis=0)
    counts = counts.reset_index()
    counts = counts.drop('index', axis=1)

    return counts, genes

In [8]:
h1299_36h, genes_h1299_36h = prep_df('raw_h1299_36h_counts.csv')

In [9]:
h1299_36h.head()

Unnamed: 0,H1299-mock-36h-A_AAAAACAATTTT,H1299-mock-36h-A_AAAACCACAAAT,H1299-mock-36h-A_AAAACCTTATCC,H1299-mock-36h-A_AAAACGCAATAA,H1299-mock-36h-A_AAAACGCCCATC,H1299-mock-36h-A_AAAACTTCTTAA,H1299-mock-36h-A_AAAACTTGCCTG,H1299-mock-36h-A_AAAAGAGGCTAT,H1299-mock-36h-A_AAAAGCAGTGAC,H1299-mock-36h-A_AAAAGCGTACGT,...,H1299-S2-36h-B_TTTGTTGAGGCA,H1299-S2-36h-B_TTTTAAATAAGC,H1299-S2-36h-B_TTTTCCTTTTCA,H1299-S2-36h-B_TTTTGCTTTAGC,H1299-S2-36h-B_TTTTGTCAATTC,H1299-S2-36h-B_TTTTGTGCTACC,H1299-S2-36h-B_TTTTTGAGTCCA,H1299-S2-36h-B_TTTTTGCAGGTT,H1299-S2-36h-B_TTTTTGCCGCTT,H1299-S2-36h-B_TTTTTTCAACTT
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,7.0,0.0,1.0,5.0,1.0,4.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2.0,2.0,0.0,1.0,3.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### в название каждой колонки после mock / s2 добавить рандомно число (0-3) чтобы создать три "повтора", затем суммировать все mock_i, s2_i


сделать с этим и с А, В группами без повторов


#### pseudo - repeats code
line = [0,1,2]
weights = [1, 2, 3] #cumulative weights 
nums = random.choices(line, cum_weights=weights, k=len(tmp.columns))
print(nums.count(0),nums.count(1),nums.count(2))

#### список рандомных групп, примерно одинаковое количество в каждой группе
tmp = h1299_36h.copy()

new_cols = []
for ind in range(len(tmp.columns)):
    new_cols.append(str(nums[ind]) + '-' + tmp.columns[ind])
    
tmp.columns = new_cols    
tmp = tmp.T.groupby([s.split('_')[0] for s in tmp.T.index.values]).sum().T

####  Посмотреть, что такое A, B

## кажется это реальные повторы!!!

In [10]:
h1299_36h = h1299_36h.T.groupby([s.split('_')[0] for s in h1299_36h.T.index.values]).sum().T
h1299_36h.head()

,H1299-S2-36h-A,H1299-S2-36h-B,H1299-mock-36h-A,H1299-mock-36h-B
0,10.0,20.0,10.0,8.0
1,2168.0,1958.0,1923.0,1304.0
2,8.0,10.0,14.0,1.0
3,820.0,692.0,761.0,549.0
4,300.0,242.0,261.0,200.0


In [36]:
h1299_36h

,H1299-S2-36h-A,H1299-S2-36h-B,H1299-mock-36h-A,H1299-mock-36h-B
0,10.0,20.0,10.0,8.0
1,2168.0,1958.0,1923.0,1304.0
2,8.0,10.0,14.0,1.0
3,820.0,692.0,761.0,549.0
4,300.0,242.0,261.0,200.0
...,...,...,...,...
15640,259.0,199.0,210.0,155.0
15641,47.0,28.0,34.0,25.0
15642,190.0,161.0,163.0,111.0
15643,1009.0,967.0,1057.0,716.0


In [11]:
counts = h1299_36h.copy()

In [12]:
pandas2ri.activate()
DESeq2 = importr("DESeq2")

In [13]:
design = pd.DataFrame({
    "experiment": [sample_name.split("-")[1] for sample_name in counts.columns],
    #"hpi": [sample_name.split("-")[2] for sample_name in counts.columns],
}, index=counts.columns)

design

,experiment
H1299-S2-36h-A,S2
H1299-S2-36h-B,S2
H1299-mock-36h-A,mock
H1299-mock-36h-B,mock


In [14]:
from rpy2.robjects import default_converter
from rpy2.robjects.conversion import rpy2py
base = importr("base")

In [67]:
def deseq(meta: pd.DataFrame, counts: pd.DataFrame, formula: str, ref: str, exp: str):
    # Calculate normalization factors
    dds = DESeq2.DESeqDataSetFromMatrix(
        countData=counts, colData=meta, design=ro.Formula(formula))
    
    dds = DESeq2.DESeq(dds) #parallel=True
    #estimateSizeFactors(dds, type = 'iterate')
    
    print(f"experiment_{exp}_vs_{ref}")
    resR = DESeq2.results(dds, name=f"experiment_{ref}_vs_{exp}")
    res = r_to_df(resR)
    res = res.sort_values("padj")
    res = res.loc[res["padj"] < 0.2]
    #res = res.loc[res["log2FoldChange"].abs() > 0.01]

    return res

In [68]:
def r_to_df(r_df):
    with localconverter(default_converter + pandas2ri.converter):
        return rpy2py(base.as_data_frame(r_df))

In [69]:
result_experiment = deseq(meta=design, counts=counts, formula="~experiment", ref="mock", exp="S2")
result_experiment

R[write to console]: converting counts to integer mode

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates

R[write to console]: mean-dispersion relationship

R[write to console]: final dispersion estimates

R[write to console]: fitting model and testing



experiment_S2_vs_mock


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
10030,8454.284303,0.635919,0.135315,4.699533,0.000003,0.040793


In [70]:
DEGs = list(result_experiment.index) # индексы диф экспр генов
gene_index = list(genes_h1299_36h) 
gene_list = []

for i in DEGs:
    gene_list.append(gene_index[int(i)])

In [71]:
result_experiment.index = gene_list

In [72]:
result_experiment

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
ENSG00000197061,8454.284303,0.635919,0.135315,4.699533,0.000003,0.040793


In [60]:
result_experiment.to_csv('h1299_36h_DEGs.csv')